In [1]:

!nvidia-smi

Sun Dec 31 06:11:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!pip install --upgrade cmake

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/u

In [3]:

!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp


Cloning into 'instant-ngp'...
remote: Enumerating objects: 4278, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 4278 (delta 69), reused 126 (delta 66), pack-reused 4100
Receiving objects: 100% (4278/4278), 187.20 MiB | 29.38 MiB/s, done.
Resolving deltas: 100% (2696/2696), done.
Submodule 'dependencies/OpenXR-SDK' (https://github.com/KhronosGroup/OpenXR-SDK.git) registered for path 'dependencies/OpenXR-SDK'
Submodule 'dependencies/args' (https://github.com/Taywee/args) registered for path 'dependencies/args'
Submodule 'dependencies/dlss' (https://github.com/NVIDIA/DLSS) registered for path 'dependencies/dlss'
Submodule 'dependencies/glfw' (https://github.com/Tom94/glfw) registered for path 'dependencies/glfw'
Submodule 'dependencies/imgui' (https://github.com/ocornut/imgui.git) registered for path 'dependencies/imgui'
Submodule 'dependencies/pybind11' (https://github.com/Tom94/pybind11) registered for path 'depende

In [4]:
compute_capability = "61" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability


env: TCNN_CUDA_ARCHITECTURES=61


In [5]:
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' NN_CONFIG_PATH

Using CutlassMLP
env: NN_CONFIG_PATH=./configs/nerf/base.json
jq: error: Could not open file NN_CONFIG_PATH: No such file or directory


In [6]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- No release type specified. Setting to 'Release'.
-- Obtained CUDA architectures from environment variable TCNN_CUDA_ARCHITECTURES=61
-- Targeting CUDA architectures: 61
CMake Warning at dependencies/tiny-cuda-n

In [7]:

!cmake --build build --config RelWithDebInfo -j `nproc`

[  2%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/pathescape.ptx
[  5%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/format.cc.o
[  8%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raystab.ptx
[ 10%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raytrace.ptx
[ 10%] Built target optix_program
[ 13%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/os.cc.o
[ 16%] Linking CXX static library libfmt.a
[ 16%] Built target fmt
[ 18%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/common_host.cu.o
[ 21%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cpp_api.cu.o
[ 24%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cutlass_mlp.cu.o
[ 27%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/encoding.cu.o
[ 29%] Building CUDA object dependencies/tiny-c

In [8]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12072 sha256=e8597ca3834e0fee231d6e7c08db57912d35c7aea8ef345044944e9386cf2015
  Stored in directory: /root/.cache/pip/wheels/7d/90/23/6358a234ca5b4ec0866d447079b97fedf9883387d1d7d074e5
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=f237d794ac2b05a667ea120209c50cd4e4557d4e0f71145c49d3d18cd98b5a06
  Stored in directory: /root/.cache/pip/wheels/29/30/94/33e8b58318aa05cb1842b365843036e0280af5983abb966b83
Successfully built commentjson lark-parser


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:

import os
scene_path = "/content/drive/MyDrive/NeRF_img_to_3d/Image-Frames/" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

In [ ]:
train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.ingp")
!python ./scripts/run.py {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path}

06:29:32 SUCCESS  Initialized CUDA 12.2. Active GPU is #0: Tesla T4 [75]
06:29:32 ERROR    Failed to load training data: Cannot load NeRF data from an empty set of paths.
Training:   0% 0/2000 [00:00<?, ?steps/s]06:29:32 INFO     Loading network config from: /content/instant-ngp/configs/nerf/base.json
06:29:32 INFO     GridEncoding:  Nmin=16 b=2 F=4 T=2^19 L=8
06:29:32 WARNING  FullyFusedMLP is not supported for the selected architecture 61. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
06:29:32 WARNING  FullyFusedMLP is not supported for the selected architecture 61. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
06:29:32 WARNING  FullyFusedMLP is not supported for the selected architecture 61. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
06:29:32 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64,layers=3)]-->1
06:29:32 

In [ ]:
video_camera_path = os.path.join(scene_path, "base_cam.json")
if not os.path.isfile(video_camera_path):
  raise FileNotFoundError(video_camera_path)

In [ ]:
video_n_seconds = 5 #@param {type:"integer"}
video_fps = 25 #@param {type:"integer"}
width = 720 #@param {type:"integer"}
height = 720 #@param {type:"integer"}
output_video_path = os.path.join(scene_path, "output_video.mp4")

!python scripts/run.py {snapshot_path} --video_camera_path {video_camera_path} --video_n_seconds 2 --video_fps 25 --width 720 --height 720 --video_output {output_video_path}
print(f"Generated video saved to:\n{output_video_path}")